In [13]:
import cv2
import numpy as np
import vehicles
import time

cnt_up=0
cnt_down=0


cap=cv2.VideoCapture("surveillance.m4v")
# cap=cv2.VideoCapture("Freewa.m4v")
# cap=cv2.VideoCapture("highway.m4v")


#Get width and height of video

w=cap.get(3)
h=cap.get(4)
frameArea=h*w
areaTH=frameArea/400

#Lines
line_up=int(2*(h/5))
line_down=int(3*(h/5))

up_limit=int(1*(h/5))
down_limit=int(4*(h/5))

print("Red line y:",str(line_down))
print("Blue line y:",str(line_up))
line_down_color=(255,0,0)
line_up_color=(255,0,255)
pt1 =  [0, line_down]
pt2 =  [w, line_down]
pts_L1 = np.array([pt1,pt2], np.int32)
pts_L1 = pts_L1.reshape((-1,1,2))
pt3 =  [0, line_up]
pt4 =  [w, line_up]
pts_L2 = np.array([pt3,pt4], np.int32)
pts_L2 = pts_L2.reshape((-1,1,2))

pt5 =  [0, up_limit]
pt6 =  [w, up_limit]
pts_L3 = np.array([pt5,pt6], np.int32)
pts_L3 = pts_L3.reshape((-1,1,2))
pt7 =  [0, down_limit]
pt8 =  [w, down_limit]
pts_L4 = np.array([pt7,pt8], np.int32)
pts_L4 = pts_L4.reshape((-1,1,2))

#Background Subtractor
fgbg=cv2.createBackgroundSubtractorMOG2(detectShadows=True)

#Kernals
kernalOp = np.ones((3,3),np.uint8)
kernalOp2 = np.ones((5,5),np.uint8)
kernalCl = np.ones((11,11),np.uint8)


font = cv2.FONT_HERSHEY_SIMPLEX
cars = []
max_p_age = 5
pid = 1


while(cap.isOpened()):
    ret,frame=cap.read()
    for i in cars:
        i.age_one()
    fgmask=fgbg.apply(frame)
    fgmask2=fgbg.apply(frame)

    if ret==True:

        #Binarization
        ret,imBin=cv2.threshold(fgmask,200,255,cv2.THRESH_BINARY)
        ret,imBin2=cv2.threshold(fgmask2,200,255,cv2.THRESH_BINARY)
        #OPening i.e First Erode the dilate
        mask=cv2.morphologyEx(imBin,cv2.MORPH_OPEN,kernalOp)
        mask2=cv2.morphologyEx(imBin2,cv2.MORPH_CLOSE,kernalOp)

        #Closing i.e First Dilate then Erode
        mask=cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernalCl)
        mask2=cv2.morphologyEx(mask2,cv2.MORPH_CLOSE,kernalCl)


        #Find Contours
        _, countours0,hierarchy=cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
        for cnt in countours0:
            area=cv2.contourArea(cnt)
            print(area)
            if area>areaTH:
                ####Tracking######
                m=cv2.moments(cnt)
                cx=int(m['m10']/m['m00'])
                cy=int(m['m01']/m['m00'])
                x,y,w,h=cv2.boundingRect(cnt)

                new=True
                if cy in range(up_limit,down_limit):
                    for i in cars:
                        if abs(x - i.getX()) <= w and abs(y - i.getY()) <= h:
                            new = False
                            i.updateCoords(cx, cy)

                            if i.going_UP(line_down,line_up)==True:
                                cnt_up+=1
                                print("ID:",i.getId(),'crossed going up at', time.strftime("%c"))
                            elif i.going_DOWN(line_down,line_up)==True:
                                cnt_down+=1
                                print("ID:", i.getId(), 'crossed going up at', time.strftime("%c"))
                            break
                        if i.getState()=='1':
                            if i.getDir()=='down'and i.getY()>down_limit:
                                i.setDone()
                            elif i.getDir()=='up'and i.getY()<up_limit:
                                i.setDone()
                        if i.timedOut():
                            index=cars.index(i)
                            cars.pop(index)
                            del i

                    if new==True: #If nothing is detected,create new
                        p=vehicles.Car(pid,cx,cy,max_p_age)
                        cars.append(p)
                        pid+=1

                cv2.circle(frame,(cx,cy),5,(0,0,255),-1)
                img=cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

        for i in cars:
            cv2.putText(frame, str(i.getId()), (i.getX(), i.getY()), font, 0.3, i.getRGB(), 1, cv2.LINE_AA)




        str_up='UP: '+str(cnt_up)
        str_down='DOWN: '+str(cnt_down)
        frame=cv2.polylines(frame,[pts_L1],False,line_down_color,thickness=2)
        frame=cv2.polylines(frame,[pts_L2],False,line_up_color,thickness=2)
        frame=cv2.polylines(frame,[pts_L3],False,(255,255,255),thickness=1)
        frame=cv2.polylines(frame,[pts_L4],False,(255,255,255),thickness=1)
        cv2.putText(frame, str_up, (10, 40), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, str_up, (10, 40), font, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        cv2.putText(frame, str_down, (10, 90), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, str_down, (10, 90), font, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.imshow('Frame',frame)

        if cv2.waitKey(1)&0xff==ord('q'):
            break

    else:
        break

cap.release()
cv2.destroyAllWindows()











Red line y: 216
Blue line y: 144
4.0
4.0
12.0
4.0
10.0
36.5
30.0
150.5
4.0
32.0
4.0
9.0
4.0
9.0
4.0
6.0
6.0
8.5
4.0
10.0
4.0
16.0
20.5
9.0
10.0
9.0
4.0
10.0
59.0
17.5
4.0
4.0
10.5
39.0
22.5
8.5
32.0
6.0
28.0
44.5
39.0
8.0
621.0
4.0
52.0
317.0
1049.5
27.0
729.0
48.5
10.0
14.0
9.0
653.5
1478.5
12.0
0.0
9.0
9.0
21.0
6.0
11.5
341.0
4.0
74.0
6.0
9.0
226.5
6.0
6.0
4.0
6.0
12.0
6.0
6.0
12.0
4.0
6.0
20.5
8.5
9.0
6.0
6.0
10.5
258.0
4.0
12.0
6.0
6.0
8.0
20.5
4.0
240.0
24.0
4.0
6.0
39.5
52.5
13.0
21.0
615.5
601.0
0.0
807.0
4.0
14.5
16.0
66.0
34.0
4.0
4.0
10.0
27.5
4.0
9.0
91.0
180.0
12.0
124.0
171.5
95.5
148.0
4.0
15.0
43.5
4.0
6.0
11.5
43.5
12.0
135.0
4.0
9.0
6.0
115.5
6.0
4.0
6.0
6.0
65.0
74.5
18.0
4.0
10.0
6.0
4.0
6.0
4.0
4.0
14.0
8.5
8.5
4.0
53.5
40.0
13.0
4.0
99.0
22.0
24.5
329.5
6.0
55.0
4.0
6.0
94.0
351.0
24.0
97.5
87.0
54.5
218.0
76.5
6.0
55.5
6.0
74.0
106.0
54.5
83.0
18.0
229.5
15.0
3329.0
125.0
161.5
8.0
30.0
134.5
126.0
6.0
10.0
4.0
10.0
12.5
18.5
4.0
8.0
4.0
14.5
12.0
12.0
9.0
8.5
117

44.0
76.0
54.5
15.0
10.0
10.0
11.5
6.0
24.0
95.5
4.0
4.0
4.0
4.0
3962.5
1129.0
562.5
559.0
41.0
44.0
76.0
54.5
4.0
10.0
10.0
9.0
6.0
21.0
95.5
4.0
4.0
4.0
3391.0
12.0
1266.0
649.5
516.0
4.0
46.5
43.5
35.0
49.5
11.0
4.0
14.0
6.0
14.5
21.5
101.0
4.0
4.0
4.0
2519.0
6.0
12.0
1307.5
735.5
554.0
4.0
30.5
47.0
52.0
16.5
11.5
4.0
17.0
39.0
20.5
41.0
88.5
4.0
1519.0
6.0
12.0
1381.0
ID: 16 crossed going up at Sun May 30 17:14:33 2021
799.0
668.5
4.0
31.5
8.5
42.5
33.5
32.0
16.5
6.0
6.0
10.0
4.0
8.5
24.5
33.5
69.5
4.0
1517.0
6.0
12.0
1381.0
799.0
678.5
4.0
31.5
8.5
42.5
33.5
4.0
16.5
6.0
10.0
10.0
4.0
8.5
24.5
33.5
69.5
4.0
1517.0
6.0
12.0
1381.0
799.0
678.5
31.5
6.0
42.5
33.5
4.0
16.5
6.0
10.0
10.0
4.0
8.5
24.5
33.5
69.5
4.0
1517.0
6.0
12.0
1381.0
799.0
678.5
31.5
6.0
42.5
33.5
4.0
16.5
6.0
6.0
10.0
8.5
24.5
33.5
58.5
4.0
16.5
4.0
9.0
1726.0
889.0
740.0
31.5
24.5
52.5
18.0
12.0
35.0
39.5
58.5
4.0
16.5
9.0
1848.5
986.5
ID: 19 crossed going up at Sun May 30 17:14:34 2021
778.5
65.5
18.0
6.0
70.0
1

1270.0
94.5
121.5
96.5
127.5
109.5
8.5
14.0
30.5
6.0
20.5
4.0
58.0
6.0
2850.5
1262.0
94.5
121.5
96.5
127.5
109.5
8.5
14.0
30.5
6.0
20.5
4.0
58.0
362.0
6.0
12.5
972.5
1665.5
120.5
122.5
75.0
140.5
96.0
19.0
23.5
27.5
11.5
22.5
6.0
48.0
42.5
18.0
17.0
1245.0
1838.5
111.5
136.5
67.5
121.0
28.0
52.5
6.0
6.0
35.5
12.0
22.0
4.0
42.0
2.5
14.0
1021.5
1637.5
151.0
84.5
302.0
8.5
20.5
4.0
4.0
34.5
14.5
20.0
4.0
29.0
2.5
14.0
1055.0
1637.5
151.0
84.5
302.0
8.5
20.5
4.0
4.0
34.5
14.5
20.0
4.0
29.0
2.5
14.0
1055.0
1637.5
151.0
84.5
302.0
8.5
20.5
4.0
4.0
34.5
14.5
20.0
4.0
29.0
2.5
14.0
1055.0
1637.5
151.0
84.5
302.0
8.5
20.5
4.0
34.5
14.5
20.0
4.0
29.0
725.0
1933.5
132.0
157.5
94.5
152.0
6.0
36.5
46.5
4.0
28.0
8.0
4.0
25.5
6.0
12.0
1963.5
138.0
95.5
365.5
8.0
32.5
11.5
8.0
29.0
11.5
20.0
2.0
4.0
2132.0
152.0
167.0
110.0
174.5
30.0
58.5
4.0
23.0
14.5
21.5
4.0
2132.0
152.0
167.0
110.0
174.5
37.0
58.5
4.0
23.0
14.5
21.5
4.0
2132.0
152.0
167.0
110.0
174.5
49.0
58.5
4.0
23.0
14.5
20.5
4.0
2132.0
152.0


3064.0
37.0
9.0
1054.0
404.0
31.0
56.5
107.5
226.5
26.5
47.5
4.0
9.0
4.0
4.0
69.0
28.0
38.0
3378.0
3064.0
37.0
9.0
1054.0
404.0
31.0
56.5
108.5
226.5
26.5
47.5
4.0
9.0
4.0
24.5
28.0
9.0
1227.5
8.5
1915.5
48.0
15.0
9.0
1042.5
708.5
85.0
26.0
87.0
109.5
51.0
30.0
94.5
4.0
8.0
8.5
14.5
28.0
9.0
251.0
6.0
1319.5
32.0
14.0
9.0
924.5
ID: 38 crossed going up at Sun May 30 17:14:41 2021
139.0
668.0
96.5
41.5
4.0
54.0
87.0
19.5
71.5
4.0
4.0
8.0
4.0
14.5
28.0
9.0
3.5
5.0
415.0
32.0
19.0
9.0
1566.5
744.0
66.5
34.0
66.0
8.0
119.5
21.5
132.5
4.0
6.0
14.5
28.0
9.0
3.5
5.0
415.0
32.0
19.0
9.0
1532.5
744.0
66.5
34.0
66.0
8.0
119.5
21.5
118.0
4.0
6.0
3.5
5.0
415.0
32.0
19.0
9.0
1532.5
744.0
66.5
34.0
66.0
8.0
119.5
21.5
108.5
4.0
6.0
3.5
5.0
415.0
18.0
6.0
19.0
4.0
1532.5
744.0
66.5
34.0
66.0
8.0
118.5
21.5
108.5
4.0
6.0
6.0
1589.0
882.5
167.0
44.5
229.0
86.0
31.0
59.5
17.5
6.0
1331.5
217.0
892.5
176.5
36.0
201.5
87.5
16.0
29.0
166.0
6.0
6.0
19.5
6.0
4.0
1983.0
964.5
96.0
37.5
199.5
93.5
6.0
26.5
12.0


1148.0
15.0
3401.0
1245.5
ID: 56 crossed going up at Sun May 30 17:14:44 2021
1143.0
78.0
53.5
184.5
14.0
165.5
54.5
44.5
6.0
22.5
156.5
1148.0
15.0
3401.0
1242.5
1136.0
74.0
53.5
184.5
14.0
165.5
54.5
44.5
6.0
22.5
156.5
1148.0
15.0
3401.0
1242.5
1135.5
68.0
53.5
184.5
14.0
165.5
54.5
44.5
6.0
22.5
156.5
1148.0
15.0
3401.0
1242.5
1135.5
68.0
53.5
184.5
14.0
165.5
54.5
44.5
6.0
22.5
156.5
27.5
3219.0
1565.5
1309.0
176.0
12.0
372.5
13.5
72.0
45.5
6.0
16.0
10.5
123.0
16.0
27.5
1250.5
893.5
2126.0
1361.5
ID: 57 crossed going up at Sun May 30 17:14:44 2021
184.0
367.0
38.0
173.5
77.5
57.5
12.0
6.0
117.5
6.0
4.0
104.0
31.0
501.5
1297.5
2272.5
1529.5
205.5
41.0
690.0
95.5
90.0
4.0
6.0
143.0
15.0
4.0
86.0
31.0
501.5
1297.5
2271.5
1529.5
201.5
41.0
689.5
95.5
90.0
6.0
143.0
15.0
4.0
86.0
31.0
501.5
1297.5
2271.0
1529.5
201.5
41.0
689.5
95.5
87.5
6.0
143.0
15.0
4.0
86.0
31.0
501.5
1297.5
2271.0
1529.5
201.5
41.0
680.5
95.5
87.5
6.0
143.0
15.0
4.0
86.0
1448.5
4.0
1163.5
2840.0
227.0
291.0
211.5


...

2875.0
227.0
2907.5
945.5
92.0
116.0
147.0
29.0
73.0
84.0
148.5
22.5
22.5
6.0
8.0
35.0
47.0
1508.5
3128.5
246.0
3265.5
575.5
86.0
92.0
232.0
157.0
25.0
60.5
119.0
122.5
23.5
21.5
4.0
10.0
29.5
48.0
1508.5
3127.5
246.0
3265.5
574.5
86.0
92.0
232.0
157.0
25.0
60.5
120.0
122.5
19.5
21.5
4.0
10.0
29.5
48.0
3968.5
4147.0
2204.5
893.0
109.5
193.0
160.5
18.5
31.0
58.5
42.5
6.0
100.0
16.0
24.5
4.0
20.0
43.5
43.0
4489.0
4770.5
2403.0
983.5
ID: 137 crossed going up at Sun May 30 17:15:10 2021
112.0
202.0
182.5
20.5
12.0
28.0
6.0
58.0
35.5
109.5
12.0
24.5
4.0
23.0
41.5
46.0
4475.5
4770.5
2393.0
983.5
112.0
202.5
182.5
20.5
12.0
28.0
58.0
56.5
109.5
12.0
24.5
4.0
23.0
41.5
46.0
4475.5
4770.5
2393.0
982.0
112.0
202.5
182.5
20.5
12.0
28.0
4.0
58.0
30.0
109.5
12.0
24.5
23.0
41.5
46.0
2176.5
4917.5
5669.0
389.0
803.0
117.5
123.0
254.0
178.0
17.0
31.0
6.0
105.5
6.0
127.0
20.0
12.0
14.0
37.0
47.0
2174.0
4917.5
5669.0
389.0
803.0
117.5
122.0
253.0
178.0
17.0
31.0
6.0
105.5
6.0
124.0
20.0
12.0
14.0
37.0
4

1172.5
ID: 149 crossed going up at Sun May 30 17:15:14 2021
558.0
484.5
67.5
57.5
181.0
4.0
47.0
6.0
4.0
31.0
31.0
91.0
3351.0
1172.5
558.0
484.5
67.5
57.5
181.0
4.0
47.0
6.0
4.0
31.0
31.0
91.0
2624.5
1235.0
581.5
513.5
71.0
45.5
156.5
8.0
28.5
49.5
6.0
4.0
29.0
34.0
96.5
2624.5
1235.0
580.5
513.5
71.0
45.5
156.5
8.0
28.5
49.5
6.0
4.0
29.0
34.0
96.5
1001.0
1419.0
672.5
575.5
52.0
126.0
40.0
209.5
59.0
6.0
6.0
31.5
34.0
95.0
789.0
1609.5
716.0
570.0
56.5
160.5
158.5
66.0
6.0
6.0
32.5
30.0
95.0
789.0
1609.5
716.0
570.0
56.5
160.5
157.5
66.0
6.0
6.0
32.5
30.0
95.0
789.0
1609.5
718.0
570.0
56.5
160.5
157.5
64.5
6.0
6.0
32.5
30.0
95.0
24.0
1729.5
750.0
606.5
67.0
127.0
18.0
211.5
60.5
6.0
32.5
32.5
79.0
24.0
1729.5
750.0
606.5
67.0
123.5
24.0
212.5
60.5
6.0
4.0
32.5
32.5
79.0
1894.0
836.5
705.0
74.0
130.5
65.0
14.5
192.0
4.0
32.5
34.0
39.5
2060.0
881.0
710.0
79.5
67.5
18.5
168.5
83.0
6.0
35.5
32.5
41.5
2060.0
881.0
710.0
53.5
67.5
17.5
168.5
81.0
6.0
37.0
32.5
41.5
2060.0
881.0
710.0
67.5
5

284.0
112.5
8.0
8.5
4.0
12.0
4.0
73.5
17.5
881.0
284.0
112.5
8.0
8.5
4.0
12.0
4.0
73.0
17.5
951.0
265.5
78.0
8.0
10.0
8.5
12.0
4.0
97.0
1109.0
399.0
41.0
6.0
10.0
25.0
4.0
14.0
6.0
115.5
1109.0
ID: 164 crossed going up at Sun May 30 17:15:19 2021
399.0
41.0
6.0
10.0
10.0
4.0
14.0
6.0
123.5
879.5
331.0
32.5
6.0
11.0
34.0
14.0
6.0
130.5
879.5
328.0
32.5
6.0
11.0
34.0
14.0
6.0
120.5
879.5
328.0
32.5
6.0
11.0
34.0
14.0
6.0
120.0
834.5
96.0
378.5
25.5
46.0
8.0
12.0
6.0
14.0
6.0
115.5
969.5
103.0
415.0
36.5
6.0
18.5
10.0
17.5
10.0
15.0
6.0
131.5
969.5
103.0
415.0
36.5
6.0
18.5
10.0
17.5
10.0
15.0
6.0
131.5
1063.5
4.0
109.5
534.0
33.0
4.0
10.0
14.0
8.0
16.0
4.0
146.5
1063.5
4.0
109.5
526.0
33.0
4.0
10.0
14.0
8.0
16.0
4.0
146.5
1061.5
4.0
109.5
526.0
33.0
4.0
10.0
14.0
8.0
16.0
4.0
146.5
1141.5
10.0
119.0
547.0
38.0
4.0
6.0
14.0
10.0
14.5
6.0
16.0
6.0
136.5
8.0
1648.5
669.5
175.5
15.0
27.0
4.0
12.0
6.0
134.5
8.0
1635.5
669.5
146.0
15.0
27.0
4.0
12.0
6.0
137.5
1670.5
632.5
173.0
6.0
6.0
6.0
14.

1515.0
401.0
451.0
649.5
214.5
155.5
44.5
4.0
12.0
14.0
1515.0
401.0
451.0
648.5
210.5
144.5
44.5
4.0
12.0
14.0
1483.0
407.0
450.5
4.0
591.5
36.0
223.5
4.0
87.5
12.0
14.0
2000.0
450.5
505.5
951.0
199.5
10.0
8.5
16.0
10.0
12.0
1796.5
527.5
524.0
932.0
32.5
203.0
6.0
8.0
8.0
1796.5
527.5
524.0
931.5
32.5
201.0
6.0
8.0
8.0
1796.5
527.5
524.0
922.5
4.0
32.5
201.0
6.0
8.0
8.0
1796.5
527.5
524.0
922.5
4.0
32.5
201.0
6.0
8.0
12.0
1595.0
255.5
593.0
549.5
997.5
86.0
195.0
111.0
6.0
10.0
2344.0
624.5
577.0
0.0
1112.0
181.0
29.5
34.5
4.0
8.0
3323.0
680.5
608.0
32.0
1219.0
79.5
171.0
3323.0
680.5
608.0
32.0
1219.0
79.5
168.0
3323.0
680.5
608.0
32.0
1219.0
79.5
168.0
2613.5
680.5
608.0
32.0
1219.0
88.5
168.0
3187.0
694.5
636.5
1130.0
75.5
75.5
37.0
83.5
153.0
4.0
4.0
3282.5
800.0
701.0
151.5
1326.5
88.0
158.5
14.0
4.0
2486.0
829.5
754.0
201.0
1559.0
42.0
100.5
144.0
14.0
8.0
4.0
2494.5
841.5
757.0
201.0
1563.0
42.0
100.5
119.5
14.0
8.0
4.0
2494.5
841.5
757.0
201.0
1563.0
42.0
100.5
119.5
14.0
8.0


8.0
2692.5
1187.0
17.5
4.0
53.0
487.5
4.0
15.5
8.0
15.5
23.5
8.0
4.0
30.5
17.0
1439.0
8.0
2692.5
1187.0
17.5
4.0
53.0
485.5
4.0
15.5
8.0
15.5
25.5
12.0
4.0
30.5
17.0
1439.0
8.0
2690.0
1187.0
17.5
4.0
53.0
474.0
4.0
15.5
8.0
15.5
25.5
12.0
4.0
30.5
17.0
1439.0
8.0
2682.0
1187.0
17.5
4.0
53.0
469.0
4.0
8.0
15.5
25.5
12.0
4.0
30.5
317.5
2967.0
1323.0
10.0
52.5
6.0
422.0
44.5
9.0
6.0
15.0
24.5
6.0
4.0
0.0
37.0
4028.5
1482.5
74.0
40.0
401.5
16.0
4.0
4.0
11.0
4.0
31.0
18.5
4.0
6.0
6.0
38.0
4538.0
1663.5
30.0
6.0
90.0
4.0
426.0
24.5
4.0
8.5
23.0
19.5
8.0
6.0
38.0
3991.5
1663.5
30.0
6.0
90.0
4.0
426.5
24.5
4.0
8.5
23.0
19.5
8.0
6.0
38.0
3976.0
1663.0
22.0
6.0
90.0
4.0
356.5
24.5
4.0
8.5
23.0
19.5
8.0
6.0
38.0
3925.0
1658.0
22.0
6.0
57.5
4.0
328.5
24.5
4.0
8.5
23.0
19.5
8.0
6.0
38.0
5327.5
1578.0
60.0
38.0
4.0
380.0
4.0
9.0
8.0
32.5
13.0
10.0
6.0
46.0
5697.0
1773.0
79.0
65.5
26.0
163.5
246.5
8.0
6.0
40.0
8.5
16.0
60.0
5649.5
1770.5
68.5
65.5
26.0
163.5
232.0
8.0
4.0
40.0
4.0
18.0
60.0
5270.5
17